<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Preparation-Part-1---NSW-Housing-Data" data-toc-modified-id="Data-Preparation-Part-1---NSW-Housing-Data-1">Data Preparation Part 1 - NSW Housing Data</a></span><ul class="toc-item"><li><span><a href="#STEP1.-Clean-Individual-Sales-and-Rent-File" data-toc-modified-id="STEP1.-Clean-Individual-Sales-and-Rent-File-1.1">STEP1. Clean Individual Sales and Rent File</a></span><ul class="toc-item"><li><span><a href="#1.1-Sales-Data" data-toc-modified-id="1.1-Sales-Data-1.1.1">1.1 Sales Data</a></span></li><li><span><a href="#1.2-Rent-Data" data-toc-modified-id="1.2-Rent-Data-1.1.2">1.2 Rent Data</a></span></li></ul></li><li><span><a href="#STEP2.-Merge-Sales-Files" data-toc-modified-id="STEP2.-Merge-Sales-Files-1.2">STEP2. Merge Sales Files</a></span></li><li><span><a href="#STEP3.-Merge-Rent-Files" data-toc-modified-id="STEP3.-Merge-Rent-Files-1.3">STEP3. Merge Rent Files</a></span></li><li><span><a href="#STEP4.-Create-Master-Stacked-Sales-&amp;-Rent-File" data-toc-modified-id="STEP4.-Create-Master-Stacked-Sales-&amp;-Rent-File-1.4">STEP4. Create Master Stacked Sales &amp; Rent File</a></span></li><li><span><a href="#STEP5:-Create-Master-Unstacked-Sales-&amp;-Rent-File" data-toc-modified-id="STEP5:-Create-Master-Unstacked-Sales-&amp;-Rent-File-1.5">STEP5: Create Master Unstacked Sales &amp; Rent File</a></span></li></ul></li></ul></div>

# Data Preparation Part 1 - NSW Housing Data

The purpose of this notebook is to:
1. Clean each individual quarterly sales file and rent files from Q3 2017 to Q1 2021 before merging all sales and rent data together.


2. Merge the cleaned sales files and the rent files into one complete housing dataset. This file will be in the **'stacked'** format, i.e., one quarter of data is stacked on top of another quarter's with multiple entry of the same postcodes in the index, and be used mainly for exploring the trends in the housing market.


3. Create the **unstacked** dataframe of the lastest 5 quarters (Q1 2020 - Q1 2021) of data for modelling, which will have the housing statistics from previous quarters on columns to be used as input variables together with demographic and economic features.

The end result of this notebook is two .csv data files:
* <i>Master_Sales_Rent_2017Q4_2021Q1.csv</i> (Stacked complete housing data Q4'17-Q1'21)
* <i>Pivot_Sales_Rent_5Quarters_SharedPOA.csv</i> (Unstacked complete housing data Q1'20-Q1'21)

Both are saved under `Files/Cleaned`.


In [1]:
import pandas as pd
import numpy as np 

import glob
import re
%matplotlib inline

## STEP1. Clean Individual Sales and Rent File

### 1.1 Sales Data

Define `salesCleanFn` function as follows:

In [2]:
def salesCleanFn(dataFolderString,osString):
    # Getting all the file names from a specified folder.
    
    #__________________________________________________________
    # If operating system is windows
    if osString == 'windows':
        dataDir = "Files\\"
        if dataFolderString[-1] != '\\':
        #if the dataFolderString does not have a forward slash, add a forward slash to the string
            fileNames = glob.glob(dataDir+dataFolderString+'\\'+'*.xlsx')
        else:
            fileNames = glob.glob(dataDir+dataFolderString+'*.xlsx')

    #__________________________________________________________
    # If operating system is mac / linux 
    else:
        dataDir = "Files/"
        if dataFolderString[-1] != '/':
            fileNames = glob.glob(dataDir+dataFolderString+'/'+'*.xlsx')
        else:
            fileNames = glob.glob(dataDir+dataFolderString+'*.xlsx')
#     print(fileNames)
    
    #__________________________________________________________
    # Looping through the fileNames to read the excel sheets. 
    frames = []
    masterDF = []

    for i, fileString in enumerate(fileNames):
        for j in range(0,8):
            df = []
            df = pd.read_excel(fileString, sheet_name="Postcode", na_values='-', header=j)

            if df.columns[0] == 'Postcode': #  checking if the 'header' parameter has the correct j value...

                # _____________________________________________________________________________________
                # adding additional columns
                regex = re.compile(r'\d+') #finds all numbers in string
                fileNumbers = regex.findall(fileString) #only works if the format of the 
                # filename is consistent. Stores the 'regex' numbers in a list.
                
                df['key'] = 's'+fileNumbers[2] # fileNumbers type = string
                year = fileNumbers[3] #type = string
                
                # the following statement searches for the month in the filename
                # if the find() function does not find the month, it returns '-1' and moves on to the next line.
                # Thus the use of !=-1. 
                if fileString.find('mar') !=-1 or fileString.find('Mar') !=-1:
                    quarter = 'Q1'
                elif fileString.find('jun') !=-1 or fileString.find('Jun') !=-1:
                    quarter = 'Q2'
                elif fileString.find('sep') !=-1 or fileString.find('Sep') !=-1:
                    quarter = 'Q3'
                elif fileString.find('dec') !=-1 or fileString.find('Dec') !=-1:
                    quarter = 'Q4'
                df['time_period'] = year + ' ' + quarter

                df['year'] = year

                df['quarter'] = quarter
                
                # some of the columns in the files are not the same, so we fix them here
                column = 'Quarterly change in Median Sales Price'
                newColumns = {'Quarterly change in Median Sales Price':'Qtly change in Median',
                             'Annual change in Median Sales Price':'Annual change in Median',
                             'Quarterly change in Count':'Qtly change in Count'}
        
                if column in df.columns:
                    df.rename(columns = newColumns, inplace= True)
                
                # finally, putting the DF into a list: frames
                frames.extend([df])
                
               


    # _____________________________________________________________________________________
    # putting all the DFs (frames) together to get a master DF
    masterDF = pd.concat(frames)
    # General cleaning
    rename_cols= {'Postcode':'postcode', 
             'Dwelling Type':'dwelling_type', 
             "First Quartile Sales Price\n$'000s" : '25%_price',
             "Median Sales Price\n$'000s" : 'median_price', 
             "Third Quartile Sales Price\n'000s" : '75%_price',
             "Mean Sales Price\n$'000s" : 'mean_price',
             'Sales\nNo.':'sales_no',
             'Qtly change in Median':'Qdelta_median',
             'Annual change in Median':'Adelta_median',
             'Qtly change in Count':'Qdelta_count',
             'Annual change in Count':'Adelta_count'}
    
    masterDF.rename(columns=rename_cols, inplace=True) #rename the columns for easier referencing



    masterDF = masterDF.drop(columns=['25%_price', '75%_price'], axis=1) # dropping unwanted columns
    
    masterDF.loc[masterDF['sales_no'].isnull(), 'sales_no'] = 5.0 #imputing NAN values. 5 is median of 0 and 10 being the 
    # range for null values in the dataset. 
    
    # fixing the NAN values in the median and mean columns 
    keys = list(masterDF['key'].unique())

    for k in keys:
    # Total
    # First, median
        k_impMedianTotal = masterDF.loc[(masterDF['median_price'].notna()) & 
                             (masterDF['dwelling_type']=='Total') &
                             (masterDF['key']==k),
                             'median_price'].median() # calculate imputer value 

        masterDF.loc[(masterDF['median_price'].isnull()) & 
                     (masterDF['dwelling_type']=='Total') &
                     (masterDF['key']==k),
                     'median_price']=k_impMedianTotal #impute

    # mean
        k_impMeanTotal = masterDF.loc[(masterDF['mean_price'].notna()) & 
                             (masterDF['dwelling_type']=='Total') &
                             (masterDF['key']==k),
                             'median_price'].median()

        masterDF.loc[(masterDF['mean_price'].isnull()) & 
                 (masterDF['dwelling_type']=='Total') &
                 (masterDF['key']==k),
                 'mean_price']=k_impMeanTotal #impute
#         print(k_impMeanTotal)
#         print('')
#         print(k)

    # Strata
    # First, median
        k_impMedianStrata = masterDF.loc[(masterDF['median_price'].notna()) & 
                             (masterDF['dwelling_type']=='Strata') &
                             (masterDF['key']==k),
                             'median_price'].median()


        masterDF.loc[(masterDF['median_price'].isnull()) & 
                     (masterDF['dwelling_type']=='Strata') &
                     (masterDF['key']==k),
                     'median_price']=k_impMedianStrata

    # mean
        k_impMeanStrata = masterDF.loc[(masterDF['mean_price'].notna()) & 
                             (masterDF['dwelling_type']=='Strata') &
                             (masterDF['key']==k),
                             'mean_price'].median()

        masterDF.loc[(masterDF['mean_price'].isnull()) & 
                     (masterDF['dwelling_type']=='Strata') &
                     (masterDF['key']==k),
                     'mean_price']=k_impMeanStrata

    # Non-Strata
    # First, median
        k_impMedianNonStrata = masterDF.loc[(masterDF['median_price'].notna()) & 
                             (masterDF['dwelling_type']=='Non Strata') &
                             (masterDF['key']==k),
                             'median_price'].median()

        masterDF.loc[(masterDF['median_price'].isnull()) & 
                     (masterDF['dwelling_type']=='Non Strata') &
                     (masterDF['key']==k),
                     'median_price']=k_impMedianNonStrata

    # mean
        k_impMeanNonStrata = masterDF.loc[(masterDF['mean_price'].notna()) & 
                             (masterDF['dwelling_type']=='Non Strata') &
                             (masterDF['key']==k),
                             'mean_price'].median()

        masterDF.loc[(masterDF['mean_price'].isnull()) & 
                     (masterDF['dwelling_type']=='Non Strata') &
                     (masterDF['key']==k),
                     'mean_price']=k_impMeanNonStrata
        continue

    masterDF.loc[masterDF['sales_no'] == 's', 'sales_no'] = 20.0 # Replace 's' with the median of 
    # 10 and 30 since there are quite a few

    masterDF['sales_no'] = masterDF['sales_no'].astype(float) # Cast type as float

    return masterDF

Next we'll call the function and save the dataframes as CSV. 


Below `old` denotes the files from **2017-2018** and `new` denotes the files from **2019-2021**. Sales files are cleaned in two batches (2017-2018, 2019-2021) because they take slightly different format in terms of the number of redundant rows before the actual table starts, and can be hard to be looped through in a single batch.

Rent files have the same issue and are cleaned in the same fashion.

In [3]:
salesOld = salesCleanFn('Sales/2017_2018', 'windows') # At this stage, if function does not run when called, may have to 
                                                    # change the second parameter to the appropriate operating system

salesOld.to_csv('Files/Cleaned/Housing/sales_2017_2018')

In [4]:
salesNew =  salesCleanFn('Sales/2019_2021', 'windows')

salesNew.to_csv('Files/Cleaned/Housing/sales_2019_2021')

The resulting data files from the above process are:
* sales_2017_2018.csv
* sales_2019_2021.csv

Both are saved in the outlier of the `Files` directory.

### 1.2 Rent Data

Define `rentCleanFn` function as follows:

In [5]:
def rentCleanFn(dataFolderString,osString):
        
    #__________________________________________________________
    # If operating system is windows
    if osString == 'windows':
        dataDir = "Files\\"
        if dataFolderString[-1] != '\\':
        #if the dataFolderString does not have a forward slash, add a forward slash to the string
            fileNames = glob.glob(dataDir+dataFolderString+'\\'+'*.xlsx')
        else:
            fileNames = glob.glob(dataDir+dataFolderString+'*.xlsx')

    #__________________________________________________________
    # If operating system is mac / linux 
    else:
        dataDir = "Files/"
        if dataFolderString[-1] != '/':
            fileNames = glob.glob(dataDir+dataFolderString+'/'+'*.xlsx')
        else:
            fileNames = glob.glob(dataDir+dataFolderString+'*.xlsx')
#     print(fileNames)
    
    #__________________________________________________________
    # Looping through the fileNames to read the excel sheets. 
    frames = []
    masterDF = []
    
    for i, fileString in enumerate(fileNames):
        for j in range(0,8):
            df = []
            df = pd.read_excel(fileString, sheet_name="Postcode", na_values='-', header=j)

            if df.columns[0] == 'Postcode': # ...if the 'header' parameter has the correct j value...

                # adding additional columns
                regex = re.compile(r'\d+') #finds all numbers in string
                fileNumbers = regex.findall(fileString)
                
                df['key'] = 'r'+fileNumbers[2] # fileNumbers type = string
    
                
        
                # some of the columns in the files are not the same, so we fix them here
                column = 'Bedroom Numbers'
                newColumns = {'Bedroom Numbers':'Number of Bedrooms'}
        
                if column in df.columns:
                    df.rename(columns = newColumns, inplace= True)
                
                
                
                frames.extend([df])# putting the DF into a list, frames

            
              



    masterDF = pd.concat(frames)
    
    # droppinig this column as we've confirmed there's an issue with the raw csv file. 
    if 'Unnamed: 10' in masterDF.columns:
        masterDF=  masterDF.drop(columns='Unnamed: 10')

    # Drop unwanted columns
    masterDF = masterDF.drop(columns=['First Quartile Weekly Rent for New Bonds\n$',
                          'Third Quartile Weekly Rent for New Bonds\n$'],
                axis=1)
    
    # Rename columns
    rename_cols= {'Postcode':'postcode',
                  'Dwelling Types':'dwelling_type', 
                  'Number of Bedrooms':'bed_number',
                  'Median Weekly Rent for New Bonds\n$': 'median_rent_newb',
                  'New Bonds Lodged\nNo.' : 'new_bonds_no',
                  'Total Bonds Held\nNo.': 'total_bonds_no',
                  'Quarterly change in Median Weekly Rent':'Qdelta_median_rent',
                  'Annual change in Median Weekly Rent':'Adelta_median_rent',
                  'Quarterly change in New Bonds Lodged':'Qdelta_new_bonds',
                  'Annual change in New Bonds Lodged':'Adelta_new_bonds'}
    
    masterDF.rename(columns=rename_cols,inplace=True)

    masterDF_ag = masterDF.loc[(masterDF['bed_number']=='Total') & (masterDF['dwelling_type']=='Total')]
    masterDF_ag = masterDF_ag.drop(columns=['bed_number','dwelling_type'], axis=1)
    
    # Impute 's' in 'new_bonds_no' and 'total_bonds_no' with 20
    masterDF_ag.loc[masterDF_ag['new_bonds_no']=='s','new_bonds_no'] = 20.0
    masterDF_ag.loc[masterDF_ag['total_bonds_no']=='s', 'total_bonds_no'] = 20.0

    # Impute na in 'new_bonds_no' and 'total_bonds_no' with 5
    masterDF_ag.loc[masterDF_ag['new_bonds_no'].isnull(),'new_bonds_no'] = 5.0
    masterDF_ag.loc[masterDF_ag['total_bonds_no'].isnull(), 'total_bonds_no'] = 5.0

    # Cast both variables as float (was object)
    masterDF_ag['new_bonds_no'] = masterDF_ag['new_bonds_no'].astype(float)
    masterDF_ag['total_bonds_no'] = masterDF_ag['total_bonds_no'].astype(float)

    # Impute na in 'median_rent' with median of the column
    masterDF_ag['median_rent_newb'].fillna(masterDF_ag['median_rent_newb'].median(), inplace=True)
    

    # Set postcode as index
    
    masterDF_ag = masterDF_ag.set_index('postcode')
    return masterDF_ag

Calling the function and saving the dataframes as CSV.

As explained in previous section, rent files are cleaned in two batches as well and saved as two separate csv files for future use.
* rent_2017_2018.csv
* rent_2019_2021.csv

In [7]:
rentNew = rentCleanFn('Rent/2019_2021', 'windows') # At this stage, if function does not run when called, may have to 
                                                    # change the second parameter to the appropriate operating system
rentNew.to_csv('Files/Cleaned/Housing/rent_2019_2021')

In [8]:
rentOld  = rentCleanFn('Rent/2017_2018', 'windows')
rentOld.to_csv('Files/Cleaned/Housing/rent_2017_2018')

## STEP2. Merge Sales Files

Merge the cleaned `sales 2017-2018` and `sales 2019-2020` from step1.

In [9]:
# Read in sales 2017-2018

sales17_18 = "Files/Cleaned/Housing/sales_2017_2018"
sales17_18 = pd.read_csv(sales17_18, usecols=['postcode', 'dwelling_type', 'median_price', 'mean_price',
                                               'sales_no', 'Qdelta_median', 'Adelta_median', 'Qdelta_count',
                                               'Adelta_count', 'key', 'time_period', 'year', 'quarter'])

# Re-arrange column order
cols = ['postcode', 'key', 'time_period', 'year', 'quarter', 
        'dwelling_type', 'median_price', 'mean_price','sales_no', 
        'Qdelta_median', 'Adelta_median', 'Qdelta_count','Adelta_count' ]

sales17_18 = sales17_18[cols]
sales17_18.head(1)

,postcode,key,time_period,year,quarter,dwelling_type,median_price,mean_price,sales_no,Qdelta_median,Adelta_median,Qdelta_count,Adelta_count
0,2000,s122,2017 Q3,2017,Q3,Total,1350.0,1516.328059,135.0,0.1345,0.4746,-0.325,-0.3112


In [10]:
# Read in sales 2019-2021

sales19_21 = "Files/Cleaned/Housing/sales_2019_2021"
sales19_21 = pd.read_csv(sales19_21, usecols=['postcode', 'dwelling_type', 'median_price', 'mean_price',
                                               'sales_no', 'Qdelta_median', 'Adelta_median', 'Qdelta_count',
                                               'Adelta_count', 'key', 'time_period', 'year', 'quarter'])
# Re-arrange column order
cols = ['postcode', 'key', 'time_period', 'year', 'quarter', 
        'dwelling_type', 'median_price', 'mean_price','sales_no', 
        'Qdelta_median', 'Adelta_median', 'Qdelta_count','Adelta_count' ]

sales19_21 = sales19_21[cols]
sales19_21.head(1)

,postcode,key,time_period,year,quarter,dwelling_type,median_price,mean_price,sales_no,Qdelta_median,Adelta_median,Qdelta_count,Adelta_count
0,2000,s128,2019 Q1,2019,Q1,Total,1160.0,1348.0,103.0,-0.0169,-0.1375,-0.1043,-0.1488


In [11]:
# Concatenate the two sales files 
sales_full = pd.concat([sales17_18, sales19_21])

# Check if all quarters are present
print(sales_full.groupby('time_period').size())

time_period
2017 Q3    1404
2017 Q4    1409
2018 Q1    1371
2018 Q2    1378
2018 Q3    1338
2018 Q4    1372
2019 Q1    1332
2019 Q2    1356
2019 Q3    1377
2019 Q4    1382
2020 Q1    1361
2020 Q2    1332
2020 Q3    1419
2020 Q4    1459
2021 Q1    1427
dtype: int64


In [12]:
# Check null values
sales_full.isnull().sum()

postcode            0
key                 0
time_period         0
year                0
quarter             0
dwelling_type       0
median_price        0
mean_price          0
sales_no            0
Qdelta_median    5857
Adelta_median    5861
Qdelta_count     5857
Adelta_count     5861
dtype: int64

The delta variables will later be removed in the actual analysis but was kept here just in case they're of any use. Hence, we were not too concerned about the nulls in them and didn't do anything to clean them up.

In [13]:
sales_full.head()

,postcode,key,time_period,year,quarter,dwelling_type,median_price,mean_price,sales_no,Qdelta_median,Adelta_median,Qdelta_count,Adelta_count
0,2000,s122,2017 Q3,2017,Q3,Total,1350.0,1516.328059,135.0,0.1345,0.4746,-0.3250,-0.3112
1,2000,s122,2017 Q3,2017,Q3,Strata,1350.0,1516.328059,135.0,0.1431,0.5553,-0.3182,-0.2703
2,2007,s122,2017 Q3,2017,Q3,Total,817.5,804.448333,36.0,-0.1090,-0.1572,-0.4462,-0.3455
3,2007,s122,2017 Q3,2017,Q3,Strata,817.5,804.448333,36.0,-0.0815,-0.1528,-0.4194,-0.3208
4,2008,s122,2017 Q3,2017,Q3,Total,995.0,1061.807024,41.0,-0.0005,0.0205,-0.1087,-0.1277


Save the complete sales dataset as .csv for later easier reference.

In [14]:
sales_full.to_csv('Files/Cleaned/Housing/Sales_2017Q3_2021Q1_Clean.csv', index=False)

## STEP3. Merge Rent Files

Merge the cleaned `rent 2017-2018` and `rent 2019-2020` from step1.

In [15]:
# Read rent 2017-2018
rent17_18 = "Files/Cleaned/Housing/rent_2017_2018"
rent17_18 = pd.read_csv(rent17_18)

# Read rent 2019-2021
rent19_21 = "Files/Cleaned/Housing/rent_2019_2021"
rent19_21 = pd.read_csv(rent19_21)

# Concat both rent files
rent_full = pd.concat([rent17_18, rent19_21])
rent_full.head()

,postcode,median_rent_newb,new_bonds_no,total_bonds_no,Qdelta_median_rent,Qdelta_new_bonds,key,Adelta_median_rent,Adelta_new_bonds
0,2000,640.0,1169.0,7914.0,-0.2000,0.5545,r121,NaN,NaN
1,2007,535.0,301.0,2231.0,-0.1371,1.1049,r121,NaN,NaN
2,2008,479.0,762.0,5020.0,-0.1812,1.3374,r121,NaN,NaN
3,2009,745.0,232.0,2966.0,-0.0067,-0.0720,r121,NaN,NaN
4,2010,650.0,805.0,8978.0,-0.0152,-0.0451,r121,NaN,NaN


In [16]:
# Check null values
rent_full.isnull().sum()

postcode                 0
median_rent_newb         0
new_bonds_no             0
total_bonds_no           0
Qdelta_median_rent    3187
Qdelta_new_bonds      3066
key                      0
Adelta_median_rent    3179
Adelta_new_bonds      3179
dtype: int64

Again, we don't really care about the null values in the delta variables as they'll later be dropped in the analysis.

In [17]:
# Check all quarters are presented
rent_full.groupby('key').size()

key
r121    612
r122    612
r123    637
r124    639
r125    638
r126    636
r127    637
r128    640
r129    641
r130    641
r131    619
r132    617
r133    617
r134    620
r135    617
dtype: int64

**NOTE:** rkey = skey-1 for the same quarter

In [18]:
# Map keys to time_periods so that can merge it with sales data later

tp = ['2017 Q3', '2017 Q4', 
      '2018 Q1', '2018 Q2', '2018 Q3', '2018 Q4', 
      '2019 Q1', '2019 Q2', '2019 Q3', '2019 Q4', 
      '2020 Q1', '2020 Q2', '2020 Q3', '2020 Q4', 
      '2021 Q1']
rkeys = ['r121','r122',
         'r123','r124','r125','r126',
         'r127','r128','r129','r130',
         'r131','r132','r133','r134',
         'r135']

rent_full['time_period'] = np.nan

for i in list(range(0,15)):
    rent_full.loc[rent_full['key']==rkeys[i], 'time_period']=tp[i]
    
rent_full.groupby('time_period').size()

time_period
2017 Q3    612
2017 Q4    612
2018 Q1    637
2018 Q2    639
2018 Q3    638
2018 Q4    636
2019 Q1    637
2019 Q2    640
2019 Q3    641
2019 Q4    641
2020 Q1    619
2020 Q2    617
2020 Q3    617
2020 Q4    620
2021 Q1    617
dtype: int64

In [19]:
# Update column name of 'key'
rent_full = rent_full.rename(columns={'key':'rkey'})


# Change columns order
cols = ['postcode', 'rkey', 'time_period', 'median_rent_newb', 'new_bonds_no', 'total_bonds_no',
       'Qdelta_median_rent', 'Qdelta_new_bonds', 'Adelta_median_rent',
       'Adelta_new_bonds']
rent_full = rent_full[cols]

rent_full.head()

,postcode,rkey,time_period,median_rent_newb,new_bonds_no,total_bonds_no,Qdelta_median_rent,Qdelta_new_bonds,Adelta_median_rent,Adelta_new_bonds
0,2000,r121,2017 Q3,640.0,1169.0,7914.0,-0.2000,0.5545,NaN,NaN
1,2007,r121,2017 Q3,535.0,301.0,2231.0,-0.1371,1.1049,NaN,NaN
2,2008,r121,2017 Q3,479.0,762.0,5020.0,-0.1812,1.3374,NaN,NaN
3,2009,r121,2017 Q3,745.0,232.0,2966.0,-0.0067,-0.0720,NaN,NaN
4,2010,r121,2017 Q3,650.0,805.0,8978.0,-0.0152,-0.0451,NaN,NaN


In [20]:
# Save full rent data into csv
rent_full.to_csv('Files/Cleaned/Housing/Rent_2017Q4_2021Q2_Clean.csv', index=False)

## STEP4. Create Master Stacked Sales & Rent File

In [21]:
# Change the name of 'key' column in the sales files from 'key' to 'skey' 
# to differentiate from 'rkey' in rent
sales_full = sales_full.rename(columns={'key':'skey'})

In [22]:
# Join the full sales data DF and the full rent data DF

sales_rent_full = pd.merge(sales_full, rent_full, how='left',
                           left_on=['postcode','time_period'],
                           right_on=['postcode', 'time_period'])

sales_rent_full.head()

,postcode,skey,time_period,year,quarter,dwelling_type,median_price,mean_price,sales_no,Qdelta_median,...,Qdelta_count,Adelta_count,rkey,median_rent_newb,new_bonds_no,total_bonds_no,Qdelta_median_rent,Qdelta_new_bonds,Adelta_median_rent,Adelta_new_bonds
0,2000,s122,2017 Q3,2017,Q3,Total,1350.0,1516.328059,135.0,0.1345,...,-0.3250,-0.3112,r121,640.0,1169.0,7914.0,-0.2000,0.5545,NaN,NaN
1,2000,s122,2017 Q3,2017,Q3,Strata,1350.0,1516.328059,135.0,0.1431,...,-0.3182,-0.2703,r121,640.0,1169.0,7914.0,-0.2000,0.5545,NaN,NaN
2,2007,s122,2017 Q3,2017,Q3,Total,817.5,804.448333,36.0,-0.1090,...,-0.4462,-0.3455,r121,535.0,301.0,2231.0,-0.1371,1.1049,NaN,NaN
3,2007,s122,2017 Q3,2017,Q3,Strata,817.5,804.448333,36.0,-0.0815,...,-0.4194,-0.3208,r121,535.0,301.0,2231.0,-0.1371,1.1049,NaN,NaN
4,2008,s122,2017 Q3,2017,Q3,Total,995.0,1061.807024,41.0,-0.0005,...,-0.1087,-0.1277,r121,479.0,762.0,5020.0,-0.1812,1.3374,NaN,NaN


In [23]:
print(sales_rent_full.shape)
print(sales_rent_full.groupby('time_period').size())

(20717, 21)
time_period
2017 Q3    1404
2017 Q4    1409
2018 Q1    1371
2018 Q2    1378
2018 Q3    1338
2018 Q4    1372
2019 Q1    1332
2019 Q2    1356
2019 Q3    1377
2019 Q4    1382
2020 Q1    1361
2020 Q2    1332
2020 Q3    1419
2020 Q4    1459
2021 Q1    1427
dtype: int64


In [24]:
print(sales_rent_full.isnull().sum())

postcode                 0
skey                     0
time_period              0
year                     0
quarter                  0
dwelling_type            0
median_price             0
mean_price               0
sales_no                 0
Qdelta_median         5857
Adelta_median         5861
Qdelta_count          5857
Adelta_count          5861
rkey                     6
median_rent_newb         6
new_bonds_no             6
total_bonds_no           6
Qdelta_median_rent    3885
Qdelta_new_bonds      3706
Adelta_median_rent    3908
Adelta_new_bonds      3908
dtype: int64


In [25]:
# Check the 6 postcodes that are null in rent:
sales_rent_full.loc[sales_rent_full['rkey'].isnull()]

,postcode,skey,time_period,year,quarter,dwelling_type,median_price,mean_price,sales_no,Qdelta_median,...,Qdelta_count,Adelta_count,rkey,median_rent_newb,new_bonds_no,total_bonds_no,Qdelta_median_rent,Qdelta_new_bonds,Adelta_median_rent,Adelta_new_bonds
1338,2817,s122,2017 Q3,2017,Q3,Total,675.25,675.250,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1339,2817,s122,2017 Q3,2017,Q3,Non Strata,712.50,743.804,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2733,2817,s123,2017 Q4,2017,Q4,Total,714.00,714.000,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2734,2817,s123,2017 Q4,2017,Q4,Non Strata,750.00,801.500,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2811,3644,s123,2017 Q4,2017,Q4,Total,475.00,484.000,20.0,0.8627,...,0.3333,-0.0769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2812,3644,s123,2017 Q4,2017,Q4,Non Strata,475.00,484.000,20.0,0.8627,...,0.3333,-0.0769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
sales_rent_full.to_csv('Files/Cleaned/Housing/Master_Sales_Rent_2017Q4_2021Q1.csv', index=False)

In [27]:
sales_rent_full.head()

,postcode,skey,time_period,year,quarter,dwelling_type,median_price,mean_price,sales_no,Qdelta_median,...,Qdelta_count,Adelta_count,rkey,median_rent_newb,new_bonds_no,total_bonds_no,Qdelta_median_rent,Qdelta_new_bonds,Adelta_median_rent,Adelta_new_bonds
0,2000,s122,2017 Q3,2017,Q3,Total,1350.0,1516.328059,135.0,0.1345,...,-0.3250,-0.3112,r121,640.0,1169.0,7914.0,-0.2000,0.5545,NaN,NaN
1,2000,s122,2017 Q3,2017,Q3,Strata,1350.0,1516.328059,135.0,0.1431,...,-0.3182,-0.2703,r121,640.0,1169.0,7914.0,-0.2000,0.5545,NaN,NaN
2,2007,s122,2017 Q3,2017,Q3,Total,817.5,804.448333,36.0,-0.1090,...,-0.4462,-0.3455,r121,535.0,301.0,2231.0,-0.1371,1.1049,NaN,NaN
3,2007,s122,2017 Q3,2017,Q3,Strata,817.5,804.448333,36.0,-0.0815,...,-0.4194,-0.3208,r121,535.0,301.0,2231.0,-0.1371,1.1049,NaN,NaN
4,2008,s122,2017 Q3,2017,Q3,Total,995.0,1061.807024,41.0,-0.0005,...,-0.1087,-0.1277,r121,479.0,762.0,5020.0,-0.1812,1.3374,NaN,NaN


##  STEP5: Create Master Unstacked Sales & Rent File

We've decided to only include housing data from the previous year or the four quarters (i.e. Q1 2020 to Q4 2020) as predictor variables for the Q1 2021 prices. Hence, in the unstacked data file we're only keeping 5 quarters (incl. Q1 2021). 

In [28]:
# Create a subset that only contains 5 Quarters of data
subset = sales_rent_full.loc[sales_rent_full['time_period'].isin(['2020 Q1','2020 Q2','2020 Q3','2020 Q4','2021 Q1'])]

# Get some of the (potentially) unnecessary variables
subset = subset.drop(columns=['Qdelta_median','Adelta_median','Qdelta_count','Adelta_count',
                              'Qdelta_median_rent', 'Adelta_median_rent','Qdelta_new_bonds','Adelta_new_bonds'],
                     axis=1)

# And only keep 'Total' dwelling type (i.e. get rid of Strata and Non-strata)
subset_total = subset.loc[subset['dwelling_type'] == 'Total']

print(subset_total.groupby('time_period').size(),'\n')
print(subset_total.groupby('dwelling_type').size(),'\n')
print(subset_total.shape)

time_period
2020 Q1    520
2020 Q2    521
2020 Q3    544
2020 Q4    563
2021 Q1    544
dtype: int64 

dwelling_type
Total    2692
dtype: int64 

(2692, 13)


In [29]:
print(subset_total.isnull().sum())

postcode            0
skey                0
time_period         0
year                0
quarter             0
dwelling_type       0
median_price        0
mean_price          0
sales_no            0
rkey                0
median_rent_newb    0
new_bonds_no        0
total_bonds_no      0
dtype: int64


In [30]:
print("number of postcode in Q4 2020:",
      subset_total.loc[subset_total['time_period']=='2020 Q4', 'postcode'].nunique())
print("number of postcode in Q1 2021:",
      subset_total.loc[subset_total['time_period']=='2021 Q1', 'postcode'].nunique())

number of postcode in Q4 2020: 563
number of postcode in Q1 2021: 544


In [31]:
pivot = subset_total.pivot_table(index='postcode', columns='time_period', values=
                                 ['median_price', 'mean_price', 'sales_no', 
                                  'median_rent_newb','new_bonds_no', 'total_bonds_no']).round(2)

pivot.head()

mean_price                                 median_price          \
time_period    2020 Q1 2020 Q2 2020 Q3 2020 Q4 2021 Q1      2020 Q1 2020 Q2   
postcode                                                                      
2000            1541.0  1322.0  1631.0  1379.0  2794.0       1225.0  1000.0   
2007             834.0   739.0   658.0   677.0   754.0        745.0   775.0   
2008             956.0  1144.0   985.0  1184.0   937.0        750.0  1173.0   
2009            1277.0  1282.0  1373.0  1661.0  1427.0        986.0  1100.0   
2010            1357.0  1395.0  1476.0  2267.0  1371.0       1280.0  1325.0   

                                     ... sales_no                          \
time_period 2020 Q3 2020 Q4 2021 Q1  ...  2020 Q1 2020 Q2 2020 Q3 2020 Q4   
postcode                             ...                                    
2000         1390.0  1110.0  1371.0  ...    105.0    74.0   100.0   155.0   
2007          655.0   651.0   763.0  ...     20.0    20.0    20.0    20.0   
2008          890.0   991.0   855.0  ...     35.0    20.0    36.0    45.0   
2009         1085.0  1075.0  1188.0  ...     20.0    32.0    56.0    75.0   
2010         1270.0  1240.0  1201.0  ...    151.0   125.0   157.0   211.0   

                    total_bonds_no                                  
time_period 2021 Q1        2020 Q1 2020 Q2 2020 Q3 2020 Q4 2021 Q1  
postcode                                                            
2000          184.0         8615.0  7595.0  8069.0  9140.0  9327.0  
2007           20.0         2116.0  1810.0  1876.0  1979.0  2024.0  
2008           49.0         4613.0  4286.0  4116.0  4514.0  4228.0  
2009           54.0         2913.0  2793.0  2913.0  3247.0  3333.0  
2010          218.0         8338.0  8079.0  8182.0  8521.0  8626.0  

[5 rows x 30 columns]

In [32]:
pivot.columns = [' '.join(col) for col in pivot.columns]
pivot.head()

,mean_price 2020 Q1,mean_price 2020 Q2,mean_price 2020 Q3,mean_price 2020 Q4,mean_price 2021 Q1,median_price 2020 Q1,median_price 2020 Q2,median_price 2020 Q3,median_price 2020 Q4,median_price 2021 Q1,...,sales_no 2020 Q1,sales_no 2020 Q2,sales_no 2020 Q3,sales_no 2020 Q4,sales_no 2021 Q1,total_bonds_no 2020 Q1,total_bonds_no 2020 Q2,total_bonds_no 2020 Q3,total_bonds_no 2020 Q4,total_bonds_no 2021 Q1
postcode,,,,,,,,,,,,,,,,,,,,,
2000,1541.0,1322.0,1631.0,1379.0,2794.0,1225.0,1000.0,1390.0,1110.0,1371.0,...,105.0,74.0,100.0,155.0,184.0,8615.0,7595.0,8069.0,9140.0,9327.0
2007,834.0,739.0,658.0,677.0,754.0,745.0,775.0,655.0,651.0,763.0,...,20.0,20.0,20.0,20.0,20.0,2116.0,1810.0,1876.0,1979.0,2024.0
2008,956.0,1144.0,985.0,1184.0,937.0,750.0,1173.0,890.0,991.0,855.0,...,35.0,20.0,36.0,45.0,49.0,4613.0,4286.0,4116.0,4514.0,4228.0
2009,1277.0,1282.0,1373.0,1661.0,1427.0,986.0,1100.0,1085.0,1075.0,1188.0,...,20.0,32.0,56.0,75.0,54.0,2913.0,2793.0,2913.0,3247.0,3333.0
2010,1357.0,1395.0,1476.0,2267.0,1371.0,1280.0,1325.0,1270.0,1240.0,1201.0,...,151.0,125.0,157.0,211.0,218.0,8338.0,8079.0,8182.0,8521.0,8626.0


In [33]:
print(pivot.shape)
print(pivot.isnull().sum())

(577, 30)
mean_price 2020 Q1          57
mean_price 2020 Q2          56
mean_price 2020 Q3          33
mean_price 2020 Q4          14
mean_price 2021 Q1          33
median_price 2020 Q1        57
median_price 2020 Q2        56
median_price 2020 Q3        33
median_price 2020 Q4        14
median_price 2021 Q1        33
median_rent_newb 2020 Q1    57
median_rent_newb 2020 Q2    56
median_rent_newb 2020 Q3    33
median_rent_newb 2020 Q4    14
median_rent_newb 2021 Q1    33
new_bonds_no 2020 Q1        57
new_bonds_no 2020 Q2        56
new_bonds_no 2020 Q3        33
new_bonds_no 2020 Q4        14
new_bonds_no 2021 Q1        33
sales_no 2020 Q1            57
sales_no 2020 Q2            56
sales_no 2020 Q3            33
sales_no 2020 Q4            14
sales_no 2021 Q1            33
total_bonds_no 2020 Q1      57
total_bonds_no 2020 Q2      56
total_bonds_no 2020 Q3      33
total_bonds_no 2020 Q4      14
total_bonds_no 2021 Q1      33
dtype: int64


Because each quarterly table has different numbers of postcodes and they're not an exact match from quarter to quarter, there will be additional null values generated after pivoting simply due to certain postcodes having data in some quarters but not in others. To serve sample size, we're going to impute the data using median.

In [34]:
pivot_fillna = pivot.fillna(pivot.median()).round(1)
pivot_fillna.isnull().sum()

mean_price 2020 Q1          0
mean_price 2020 Q2          0
mean_price 2020 Q3          0
mean_price 2020 Q4          0
mean_price 2021 Q1          0
median_price 2020 Q1        0
median_price 2020 Q2        0
median_price 2020 Q3        0
median_price 2020 Q4        0
median_price 2021 Q1        0
median_rent_newb 2020 Q1    0
median_rent_newb 2020 Q2    0
median_rent_newb 2020 Q3    0
median_rent_newb 2020 Q4    0
median_rent_newb 2021 Q1    0
new_bonds_no 2020 Q1        0
new_bonds_no 2020 Q2        0
new_bonds_no 2020 Q3        0
new_bonds_no 2020 Q4        0
new_bonds_no 2021 Q1        0
sales_no 2020 Q1            0
sales_no 2020 Q2            0
sales_no 2020 Q3            0
sales_no 2020 Q4            0
sales_no 2021 Q1            0
total_bonds_no 2020 Q1      0
total_bonds_no 2020 Q2      0
total_bonds_no 2020 Q3      0
total_bonds_no 2020 Q4      0
total_bonds_no 2021 Q1      0
dtype: int64

In [35]:
# Reset index so that postcodes will be saved as a variable
pivot_fillna.reset_index(inplace=True)

# Save the output as a csv
pivot_fillna.to_csv('Files/Cleaned/Housing/Pivot_Sales_Rent_5Quarters_Imputed.csv', index=False)